# 合成音声の生成
- ここでは，目的話者の短い音声を参照として，その人の話者性を持つ任意の文章をの発話を生成する方法を概説します．
- この技術はzero-shot voice cloningと呼ばれます．
- ここでは，[CosyVoice](https://github.com/FunAudioLLM/CosyVoice/tree/main)というモデルを用いて合成音声を生成する方法を示します．

## 環境構築
このノートブックを実行する前に、CosyVoice用の環境をセットアップしてください：

In [ ]:
!uv sync --extra cosyvoice

## コードのダウンロード
CosyVoiceのコードをクローンします．

In [ ]:
# CosyVoiceをクローン
!git clone --recursive https://github.com/FunAudioLLM/CosyVoice.git

## モデル本体をダウンロード

In [ ]:
!mkdir -p pretrained_models

# # Hugging Faceからモデルをダウンロード
# !git clone https://huggingface.co/FunAudioLLM/CosyVoice2-0.5B pretrained_models/CosyVoice2-0.5B

# modelscopeからDL
from modelscope import snapshot_download
snapshot_download('iic/CosyVoice2-0.5B', local_dir='pretrained_models/CosyVoice2-0.5B')

## 実行

Qwen2ForCausalLMが見つからないと言われたり，何もしていないのにエラーが出たときは，ipynbのカーネルを再起動してみてください．
一応重いけどCPUでも動くはず．

In [ ]:
import sys

sys.path.append("CosyVoice")
sys.path.append("CosyVoice/third_party/Matcha-TTS")
from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
import torchaudio
import os

# modelのインスタンス化
cosyvoice = CosyVoice2(
    "pretrained_models/CosyVoice2-0.5B",
    load_jit=False,
    load_trt=False,
    load_vllm=False,
    fp16=False,
)

# 参照音声を読み込み
prompt_speech_16k = load_wav("./audiofile/ymgt.wav", 16000)

# 合成音声を生成
## 読み上げたい文章と，参照音声の文字起こしをそれぞれ指定
## うまく出力されないときはこれらを調整する．日本語は[breath]を入れると安定しがち．．
## 本来は特殊トークンを入れなくても動くらしいが，入れたほうが安定するので入れています．
## 参考：公式Usage：https://github.com/FunAudioLLM/CosyVoice/blob/main/README.md
## 参考：特殊トークン: https://github.com/FunAudioLLM/CosyVoice/blob/main/cosyvoice/tokenizer/tokenizer.py
reading_texts = [
    "<|jp|> [breath] 南おおさわキャンパスの法定停電に伴い、以下の期間はキャンパススクエア・自動証明書発行機を利用できません。",
    "<|en|> An extremely fast Python package and project manager, written in Rust.",
    "<|jp|> [breath] バーデン＝ヴュルテンベルク州の州都はシュトゥットガルトである",
]
prompt = "<|jp|> オッケーグーグル，今日の予定を教えて"

out_dir = "./CosyVoice_output"
os.makedirs(out_dir, exist_ok=True)

for i, txt in enumerate(reading_texts):
    # outputは一つだけだが，返り値のType的にforで拾ってやる必要がある（他にもやり方はあるが公式Usageに則る）
    for output in cosyvoice.inference_zero_shot(
        txt,
        prompt,
        prompt_speech_16k,
        stream=False,
        text_frontend=False,
    ):
        cloned_speech = output["tts_speech"]

        # 音声を保存
        torchaudio.save(
            f"{out_dir}/out{i}.wav",
            src=cloned_speech,
            sample_rate=cosyvoice.sample_rate,
        )

## 音声の確認

In [ ]:
import IPython.display as ipd

# 参照音声を再生
print("参照音声:")
display(ipd.Audio("./audiofile/ymgt.wav"))

# 生成された音声を再生
print("\n生成された音声1 (日本語):")
display(ipd.Audio(f"{out_dir}/out0.wav"))

print("\n生成された音声2 (英語):")
display(ipd.Audio(f"{out_dir}/out1.wav"))

print("\n生成された音声3 (日本語):")
display(ipd.Audio(f"{out_dir}/out2.wav"))